# Load Needed Packages

In [1]:
import numpy as np

from enterprise_extensions import blocks
from enterprise.signals import signal_base, gp_signals
from enterprise.signals import gp_priors as gpp
from enterprise.signals import parameter
from enterprise_extensions.model_utils import get_tspan
from enterprise_extensions import models
import scipy.linalg as sl

import pickle, json, os, corner, glob, random, copy, time, inspect

%load_ext autoreload
%autoreload 2
# %load_ext line_profiler

# Choose a data set

In [2]:
datadir = '/home/koonima/FAST/Data/Pickle/'
with open(datadir + f'v1p1_de440_pint_bipm2019.pkl', 'rb') as fin:
    psrs = pickle.load(fin)
psrlist = [psr.name for psr in psrs]
with open(datadir + f'v1p1_all_dict.json', 'r') as fin:
    noise_dict = json.load(fin)

libstempo not installed. PINT or libstempo are required to use par and tim files.


## Frequency-bins

In [3]:
Tspan = get_tspan(psrs) # The time-span of the entire PTA
crn_bins = 30 # number of frequency-bins for the GWB
int_bins = 30 # number of frequency-bins for the non-GWB (IRN) red noise
assert int_bins >= crn_bins
common_psd = 'broken_powerlaw'
psd_func = gpp.broken_powerlaw
common_gamma = None
orf = 'crn'
orf_name ='crn'

In [4]:
pta = models.model_general(psrs, 
                    tm_var=False, 
                    tm_linear=True,
                    tmparam_list=None,
                    tm_svd=True, 
                    tm_norm=True,
                    noisedict=noise_dict, 
                    white_vary=False,
                    Tspan=Tspan, 
                    modes=None, 
                    wgts=None, 
                    logfreq=False,
                    nmodes_log=10,
                    common_psd= common_psd, 
                    common_components=crn_bins,
                    tnequad=False,
                    log10_A_common=None, 
                    gamma_common=common_gamma,
                    common_logmin=None, 
                    common_logmax=None,
                    orf=orf, 
                    orf_names=orf_name, 
                    orf_ifreq=0, 
                    leg_lmax=5,
                    upper_limit_common=None, 
                    upper_limit=False,
                    red_var=True, 
                    red_psd='powerlaw', 
                    red_components=int_bins, 
                    upper_limit_red=None,
                    red_select=None, 
                    red_breakflat=False, 
                    red_breakflat_fq=None,
                    bayesephem=False,
                    be_type='setIII_1980', 
                    is_wideband=False, 
                    use_dmdata=False,
                    dm_var=False, 
                    dm_type='gp', 
                    dm_psd='powerlaw', 
                    dm_components=30,
                    upper_limit_dm=None, 
                    dm_annual=False, 
                    dm_chrom=False, 
                    dmchrom_psd='powerlaw',
                    dmchrom_idx=4, 
                    gequad=False, 
                    coefficients=False, 
                    pshift=False,
                    select='backend', 
                    tm_marg=True, 
                    dense_like=True,
                    delta_common=0.)

In [10]:
chosen_psd_model_params = np.array(
    [
        f'gw_{orf_name}_' + str(_)
        for _ in inspect.signature(psd_func).parameters
    ]
)
chosen_psd_model_params = [str(_) for _ in chosen_psd_model_params if _ in '\t'.join(pta.param_names)]
pnames = [pname for pname in pta.param_names[:2 * len(psrs)]]
[pnames.append(str(_)) for _ in chosen_psd_model_params]
pnames

['B1855+09_red_noise_gamma',
 'B1855+09_red_noise_log10_A',
 'B1937+21_red_noise_gamma',
 'B1937+21_red_noise_log10_A',
 'B1953+29_red_noise_gamma',
 'B1953+29_red_noise_log10_A',
 'J0023+0923_red_noise_gamma',
 'J0023+0923_red_noise_log10_A',
 'J0030+0451_red_noise_gamma',
 'J0030+0451_red_noise_log10_A',
 'J0340+4130_red_noise_gamma',
 'J0340+4130_red_noise_log10_A',
 'J0406+3039_red_noise_gamma',
 'J0406+3039_red_noise_log10_A',
 'J0437-4715_red_noise_gamma',
 'J0437-4715_red_noise_log10_A',
 'J0509+0856_red_noise_gamma',
 'J0509+0856_red_noise_log10_A',
 'J0557+1551_red_noise_gamma',
 'J0557+1551_red_noise_log10_A',
 'J0605+3757_red_noise_gamma',
 'J0605+3757_red_noise_log10_A',
 'J0610-2100_red_noise_gamma',
 'J0610-2100_red_noise_log10_A',
 'J0613-0200_red_noise_gamma',
 'J0613-0200_red_noise_log10_A',
 'J0636+5128_red_noise_gamma',
 'J0636+5128_red_noise_log10_A',
 'J0645+5158_red_noise_gamma',
 'J0645+5158_red_noise_log10_A',
 'J0709+0458_red_noise_gamma',
 'J0709+0458_red_nois

In [12]:
indices = []
temp_list1 = pta.param_names[:]  # Create a copy to avoid modifying the original list
for element in pnames:
    index = temp_list1.index(element)
    indices.append(index)
    temp_list1[index] = None
indices = np.array(indices)

In [ ]:
priors = np.array([str(pname) for pname in pta.params])[indices]
priors

array(['B1855+09_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'B1855+09_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'B1937+21_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'B1937+21_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'B1953+29_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'B1953+29_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'J0023+0923_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'J0023+0923_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'J0030+0451_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'J0030+0451_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'J0340+4130_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'J0340+4130_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'J0406+3039_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'J0406+3039_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)',
       'J0437-4715_red_noise_gamma:Uniform(pmin=0, pmax=7)',
       'J0437-4715_red_noise_log10_A:Uniform(pmin=-20, 

In [32]:
# save paramter list
with open(os.path.join('./', "pars.txt"), "w") as fout:
    for pname in pnames:
        fout.write(pname + "\n")

# save paramter list
with open(os.path.join('./', "priors.txt"), "w") as fout:
    for pname in priors:
        fout.write(pname + "\n")

In [ ]:
savepath = './'
os.makedirs(savepath, exist_ok=False)
if orf == 'crn':
    np.save(savepath + '/TNT.npy', np.array(pta.get_TNT(params={})))
    np.save(savepath + '/TNr.npy', np.array(pta.get_TNr(params={}))[..., None])
else:
    np.save(savepath + '/TNT.npy', sl.block_diag(*pta.get_TNT(params={})))
    np.save(savepath + '/TNr.npy',  np.concatenate(pta.get_TNr(params={})))